In [84]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta 

df = pd.DataFrame()
os.getcwd()
 

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [682]:
# pd.read_excel('./data/Coach11-1to11-5.xlsx', sheet_name=0)\
#     .to_csv('./data/coach_15.csv', encoding = 'utf_8_sig', index=None)
coach = pd.read_csv('./data/coach_15.csv')
coach = coach.dropna(axis=1, how = 'all')
# task_collect = coach.groupby(['姓名', '开始日期'])['任务类型'].apply(list).astype(str).reset_index();task_collect
# onoff = task_collect[task_collect['任务类型'].str.contains('(下班.+上班)')][['姓名','开始日期']].sum(axis = 1)
# coach['flag'] = coach[['姓名','开始日期']].sum(axis = 1)
coach['开始时间'] = pd.to_datetime(coach['开始时间'])
coach['结束时间'] = pd.to_datetime(coach['结束时间'])

# coach_nopunch = coach[~coach['flag'].isin(onoff)]
# coach_dopunch = coach[ coach['flag'].isin(onoff)]
coach['sprm'] = (60/coach['SPR标准数量']).replace([np.inf, -np.inf], 0)

wh = coach.groupby(['姓名','开始日期']).agg(
    {
        '开始时间': 'min',
        '结束时间': 'max',
        'sprm': 'sum',
    }
).reset_index()
wh['work_hour'] = wh['结束时间'] - wh['开始时间']
wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')
coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']

# 标准时长计算
coach = pd.concat([coach, pd.DataFrame(list(coach['调整后持续时间'].str.split(':')))], axis =1).drop('序号',axis =1)
coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
coach[[0,1,2]] = coach[[0,1,2]].astype(int)
coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
coach = coach.drop([0,1,2], axis = 1)

In [446]:
# coach[coach['姓名'] == '孙清元']

In [428]:
# plot_data('2021-11-05').info()
# coach['开始日期'].unique()
# array(['2021-11-05', '2021-11-04', '2021-11-03', '2021-11-02',
#        '2021-11-01'], dtype=object)

In [86]:
def plot_data(riqi):
    data = coach[coach['开始日期'] == riqi][['姓名', '工作组', '直属上级', '任职时间', '职位',\
    'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
    'work_hour_in_hour', 'sprm_perhour']].drop_duplicates()
    data['工作组'] = data['工作组'].fillna('unknow')
    data['工作组'] = data['工作组'].fillna('unknow')

    return data
def plot_(riqi):
    fig2 = px.scatter(
        data_frame = plot_data(riqi), x = '姓名', y= 'sprm_perhour', hover_data=['工作组' ],
        size = 'work_hour_in_hour', color = '工作组', 
        # facet_col = '开始日期', 
        title= "报告日期" + riqi)  
    # hover_data=  ['account_name_en'] , 


    # fig2.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 1)
    # fig2.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 2)

    return fig2

plot_('2021-11-03')

KeyError: '开始日期'

In [283]:
# SELECT  * FROM dm_dsc_smart.dwd_task
# where substr(station_name, 1,4) = 'COAC'
# and work_group_id != 0 and work_group_name != ''
# and inc_day = '20211127'
 

In [534]:
# coach = coach.dropna(how = 'all', axis = 1)
    # time_convert()
def time_convert(col):
    coach[col] = coach[col].astype(int)
    coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
    return coach
for i in ['start_time', 'end_time', 'hire_time', 'update_time']:
    time_convert(i)
coach['start_time'] = pd.to_datetime(coach['start_time'])
coach['end_time']   = pd.to_datetime(coach['end_time'])
coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)


# wh = coach.groupby(['worker_name','start_date']).agg(
#         {
#             'start_time': 'min',
#             'end_time': 'max',
#             'sprm': 'sum',
#         }
#     ).reset_index()
# wh['work_hour'] = wh['结束时间'] - wh['开始时间']
# wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
# wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
# wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
# coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')

In [541]:
coach[coach[['duration',
    'adjustment_duration', 'adjusted_duration', ]].sum(axis = 1) != 0]

,station_name,worker_name,work_group_name,up_worker_name,hire_time,worker_post_name,worker_level_name,work_num,start_time,end_time,...,min_operate_volume,work_content,work_content_refer,work_content_is_measure,start_date,end_date,report_date,create_time,update_time,sprm
4052,COACH SHA WGQ WHS,孙孟孟,NaN,Michelle Wu,1970-01-01 08:00:00,操作员,操作员,0.0,2021-11-27 06:25:02,2021-11-27 06:25:13,...,0.0,叉车安全检查,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,2021-11-30 08:13:11,0.0
4053,COACH SHA WGQ WHS,孙孟孟,NaN,Michelle Wu,1970-01-01 08:00:00,操作员,操作员,0.0,2021-11-27 06:25:13,2021-11-27 15:25:02,...,0.0,无效时间,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,2021-11-30 08:13:11,0.0


In [511]:
# # coach= coach.drop(['dwd_task_id', 'station_id', 'work_turn_id', 'raw_data'], axis  =1)

# coach['work_content'].value_counts()
# coach['work_content_refer'].value_counts()
# coach

In [477]:
# %%time
# df = pd.DataFrame()
# for i in np.arange(coach.shape[0]):
#     df = pd.concat([pd.DataFrame.from_dict(json.loads(coach['raw_data'][i]), orient="index" ).T, df], axis = 0)

In [698]:
# coach[coach['start_date'] == '2021-11-23'] 
# coach

In [663]:
def plot0_date(date, col):
    # sns.histplot(
    return coach[coach['start_date'] == date][col].value_counts()

In [673]:
plot0_date('2021-11-28', 'work_content_refer')
# pd.set_option("display.max_rows", 15, "display.max_columns", None)
# coach.head()

0      5614
360     786
15      143
Name: work_content_refer, dtype: int64

In [254]:
coach = pd.read_csv('./data/coach1129_1202.csv', sep = '\001')
coach = coach.dropna(how = 'all', axis = 1)
coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
coach = coach.dropna(how = 'all', axis = 1)

In [188]:
# coach.to_csv('./data/coach_cleaned_drop_0.csv', encoding = 'utf_8_sig')

In [282]:
def data_prepare(coach):

    """
    time unix convert, 
    转换后标准时长换算成秒,
    sprm 计算.
    工作在勤时间
    """
    coach = pd.read_csv('./data/coach1129_1202.csv', sep = '\001')
    coach = coach.dropna(how = 'all', axis = 1)
    coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
    # coach = coach.dropna(how = 'all', axis = 1)
    # coach = coach.drop('raw_data', axis = 1)
    # time_convert()
    def time_convert(col):
        coach[col] = coach[col].astype(int)
        coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
        return coach
    for i in ['start_time', 'end_time', 'hire_time']:
        time_convert(i)
    coach['start_time'] = pd.to_datetime(coach['start_time'])
    coach['end_time']   = pd.to_datetime(coach['end_time'])
    # coach['duration'] =  coach['end_time'] - coach['start_time']
    coach = coach[~coach['work_content'].isna()]
    coach = coach[coach['work_content'] != '无效时间']
    coach = coach[coach['worker_post_name'] != '操作经理']
    # sprm calculation
    coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)
   
    wh = coach.groupby(['worker_name','inc_day']).agg(
        {
            'start_time': 'min',
            'end_time': 'max',
            'sprm': 'sum',
        }
    ).reset_index()
    
    wh['work_hour'] = wh['end_time'] - wh['start_time']
    wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
    wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
    wh = wh.drop(['end_time','start_time', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
    coach = coach.merge(wh, on = ['worker_name', 'inc_day'], how = 'inner')
    # coach = coach[coach['duration'] != '0']
    coach = coach[coach['sprm'] != 0]
    coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']
    """
    计算转换后时间长度, 换算成 
    秒
    """
    # coach = pd.concat([coach, pd.DataFrame(list(coach['adjusted_duration'].str.split(':')))], axis =1) 
    # coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    # coach[[0,1,2]] = coach[[0,1,2]].astype(int)
    # coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
    # coach = coach.drop([0,1,2], axis = 1) 
    
    return coach
coach = data_prepare(coach)


In [183]:
coach = coach[coach['work_content'] != '无效时间']
coach = coach[coach['worker_post_name'] != '操作经理']

In [184]:
coach = coach.sort_values(['worker_name', 'start_time', 'end_time'])
shift = coach[['worker_name', 'start_time', 'end_time']]
shift = shift.groupby(['worker_name']).shift(1)
shift.columns = ['start_shift1', 'end_shift1']

In [185]:
test = pd.concat([coach, shift], axis = 1)
test['gap'] = test['start_time'] - test['end_shift1'] 

In [186]:
test[test['gap'] > '0 days 00:00:00']

,dwd_task_id,account_id,account_name,station_id,station_code,station_name,worker_id,worker_name,work_turn_id,work_turn_name,work_group_id,work_group_name,up_worker_id,up_worker_name,hire_time,worker_post_id,worker_post_name,worker_level_id,worker_level_type,worker_level_name,work_num,start_time,end_time,duration,adjustment_duration,adjusted_duration,sku_no,min_package_quantity,min_operate_length,min_operate_width,min_operate_height,min_operate_weight,min_operate_volume,flow,action,work_content_id,work_content,work_content_type,work_content_is_measure,work_content_refer,productive_time,work_content_unit,start_date,end_date,report_date,source,updater_id,is_cancel,efficiency,orig_task_type,orig_user_id,create_time,update_time,pallet_operation,inc_day,sys,sprm,start_shift1,end_shift1,gap
35812,10005659-19771-0-20211126-1637912222-2,10469768,Zhenhao Yang,10005659,NaN,COACH SHA WGQ WHS,19771,Zhenhao Yang,0,NaN,0,NaN,19768,Michelle Wu,1970-01-01 08:00:00,10109369,OE协调员,10039149,30000001,操作主管,0.0,2021-11-27 00:37:02,2021-11-27 00:37:02,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,10121891,下班,4,0,0,NaN,NaN,2021-11-27,2021-11-27,2021/11-27,7,0,0,0.0,NaN,NaN,1638300996,2021-12-01 03:36:36,0,20211126,ptp,0.0,2021-11-26 15:37:02,2021-11-26 15:37:16,0 days 08:59:46
21221,10005659-19774-10006648-20211122-1637541282-2,10469982,丁叶,10005659,NaN,COACH SHA WGQ WHS,19774,丁叶,10006648,白班,0,NaN,19770,Carlo Dai,1970-01-01 08:00:00,10210780,组长,10039079,20000001,组长,0.0,2021-11-22 17:30:00,2021-11-22 17:30:00,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,10121891,下班,4,0,0,NaN,NaN,2021-11-22,2021-11-22,2021/11-22,7,0,0,0.0,NaN,NaN,1638192585,2021-11-29 21:29:45,0,20211122,ptp,0.0,2021-11-22 08:34:42,2021-11-22 08:34:51,0 days 08:55:09
45809,wmos-wmos-264015431,10469982,丁叶,10005659,CN-085,COACH SHA WGQ WHS,19774,丁叶,0,NaN,10025988,收货组,19770,Carlo Dai,2012-11-20 00:00:00,10210780,组长,10039079,20000001,组长,1.0,2021-11-27 09:14:02,2021-11-27 09:15:10,0,0,0,79468 B4PE2,1.0,440.0,320.0,150.0,1600.0,21120000.0,NaN,NaN,0,NaN,0,0,0,NaN,NaN,2021-11-27,2021-11-27,2021-11-27,6,0,0,0.0,600.0,DSCSHAP2369,1638300996,2021-12-01 03:36:36,0,20211127,wms,0.0,2021-11-22 17:30:00,2021-11-22 17:30:00,4 days 15:44:02
49705,wmos-wmos-264015724,10469982,丁叶,10005659,CN-085,COACH SHA WGQ WHS,19774,丁叶,0,NaN,10025988,收货组,19770,Carlo Dai,2012-11-20 00:00:00,10210780,组长,10039079,20000001,组长,3.0,2021-11-27 09:15:14,2021-11-27 09:15:34,0,0,0,76128 B4/BK,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,NaN,0,0,0,NaN,NaN,2021-11-27,2021-11-27,2021-11-27,6,0,0,0.0,600.0,DSCSHAP2369,1638300996,2021-12-01 03:36:36,0,20211127,wms,0.0,2021-11-27 09:14:02,2021-11-27 09:15:10,0 days 00:00:04
47356,wmos-wmos-264015997,10469982,丁叶,10005659,CN-085,COACH SHA WGQ WHS,19774,丁叶,0,NaN,10025988,收货组,19770,Carlo Dai,2012-11-20 00:00:00,10210780,组长,10039079,20000001,组长,2.0,2021-11-27 09:15:43,2021-11-27 09:16:03,0,0,0,2648 SVA47,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0,NaN,0,0,0,NaN,NaN,2021-11-27,2021-11-27,2021-11-27,6,0,0,0.0,600.0,DSCSHAP2369,1638300996,2021-12-01 03:36:36,0,20211127,wms,0.0,2021-11-27 09:15:14,2021-11-27 09:15:34,0 days 00:00:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,wmos-wmos-265515827,11018843,黄义林,10005659,CN-085,COACH SHA WGQ WHS,25611,黄义林,0,NaN,10025988,收货组,0,NaN,2021-03-04 00:00:00,10109295,操作员,10038968,10000001,操作员,25.0,2021-11-30 14:55:12,2021-11-30 14:55:42,0,0,0,C1569 IMBLK,1.0,290.0,190.0,80.0,630.0,4408000.0,NaN,NaN,0,NaN,0,0,0,NaN,NaN,2021-11-30,2021-11-30,2021-11-30,6,0,0,0.0,200.0,ADC026RF,1638300996,2021-12-01 03:36:36,0,20211130,wms,0.0,2021-11-30 14:42:00,2021-11-30 14:42:09,0 days 00:13:03
30191,wmos-wmos-265517351,11018843,黄义林,10005659,CN-085,COACH SHA WGQ WHS,25611,黄义林,0,NaN,10025988,收货组,0,NaN,2021-03-04 00:00:00,10109295,操作员,10038968,10000001,操作员,35.0,2021-11-30 14:56:53,2021-11-30 14:57:36

In [265]:
test = coach[(coach['inc_day'].astype(str).str.contains('20211130')) & (coach['worker_name'] == '张现花')]

In [266]:
def time_convert(coach,col):
    coach[col] = coach[col].astype(int)
    coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
    return coach
for i in ['start_time', 'end_time', 'hire_time']:
    time_convert(test,i)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_14824/3798527186.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_14824/3798527186.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [280]:
pd.to_datetime(test['end_time']).min()

Timestamp('2021-11-30 08:34:37')

In [279]:
pd.to_datetime(test['end_time']).max() - pd.to_datetime(test['end_time']).min()

Timedelta('0 days 16:49:03')

In [267]:
test[test['start_time'] == '2021-11-30 08:57:23'] 

,account_name,station_id,station_code,station_name,worker_id,worker_name,work_group_id,work_group_name,up_worker_id,up_worker_name,hire_time,worker_post_id,worker_post_name,work_num,start_time,end_time,duration,adjustment_duration,adjusted_duration,sku_no,work_content,work_content_type,work_content_is_measure,work_content_refer,start_date,end_date,report_date,inc_day


In [242]:
wh = test.groupby(['worker_name','inc_day']).agg(
        {
            'start_time': 'min',
            'end_time': 'max',
            'sprm': 'sum',
        }
    ).reset_index()
    
wh['work_hour'] = wh['end_time'] - wh['start_time']

In [250]:
wh 

,worker_name,inc_day,start_time,end_time,sprm,work_hour
0,郁黄祥,20211130,2021-11-30 12:52:54,2021-12-01 00:09:52,6720.0,0 days 11:16:58


In [253]:
[i.total_seconds() for i in wh['work_hour']][0]/3600

11.282777777777778

In [301]:
# coach[(coach['start_time'].astype(str).str.contains('2021-11-30')) & (coach['worker_name'] == '郁黄祥')]

In [232]:
coach[(coach['worker_name'] == '郁黄祥') & (coach['start_date'] == '2021-11-30')]['start_time'].min()

Timestamp('2021-11-30 12:52:54')

In [234]:
coach[(coach['worker_name'] == '郁黄祥') & (coach['start_date'] == '2021-12-01')]

,account_name,station_id,station_code,station_name,worker_id,worker_name,work_group_id,work_group_name,up_worker_id,up_worker_name,hire_time,worker_post_id,worker_post_name,work_num,start_time,end_time,duration,adjustment_duration,adjusted_duration,sku_no,work_content,work_content_type,work_content_is_measure,work_content_refer,start_date,end_date,report_date,inc_day,sprm,SPRM_total_of_day,work_hour_in_min,work_hour_in_hour,sprm_perhour


In [ ]:
# =======

In [284]:
coach_out = coach[['worker_name',  'work_group_name', 'up_worker_name', 'hire_time',\
         'worker_post_name', 'SPRM_total_of_day', 'work_hour_in_min',
         'work_hour_in_hour', 'sprm_perhour', 'station_name', 'inc_day']].drop_duplicates()
coach_out['inc_day'] = coach_out['inc_day'].astype(str)
        #  'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
        #  'work_hour_in_hour', 'sprm_perhour',

In [300]:
pd_loaddata.pd_tocsv(coach_out, './data/coach_1202_outdata.csv' )

In [298]:
from fyenn_class import pd_loaddata
pd_loaddata.show_line(15,None)
# coach[(coach['account_name'] == '孙清元') & (coach['start_date'] == '2021-11-30')].sort_values('start_time')

In [303]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_data(coach, riqi, site):
    data = coach[coach['station_name'].str.contains(site)]
    data = coach[coach['inc_day'] == riqi].drop_duplicates()
    data['work_group_name'] = data['work_group_name'].fillna('unknown')
    return data.drop_duplicates()
def plot_(coach, riqi, site):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    data = plot_data(coach, riqi, site)
    fig.add_trace(go.Bar(
        x = data['worker_name'], y = data['sprm_perhour'], name = 'sprm_perhour'
        ),
        secondary_y = False)
    fig.add_trace(go.Scatter(x = data['worker_name'], y = data['work_hour_in_hour'], name = 'working_hour'),\
        secondary_y = True)
    # fig.update_layout(
    #         title=go.layout.Title(text="工时 & sprm/h; 时间:" + riqi + '\t'+ coach['station_name'][0])
    #         )
    fig.update_yaxes(title_text="<b>sprm/h</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>working_hour</b>", secondary_y=True)
    return fig

plot_(coach_out, '20211130', 'COA')

In [305]:
plot_(coach_out, '20211130','COA').write_html('./data/c20211130.html')